In [1]:
import torch
from prepare_data import create_or_load
from collator import T2TDataCollator
from transformers import AdamW, get_scheduler, Trainer, TrainingArguments, Adafactor
from transformers import T5Tokenizer
from model import T5PromptTuningLM
from transformers.data.processors.squad import SquadV2Processor, squad_convert_examples_to_features

In [2]:
model_name = 't5-small'
n_tokens = 10
batch_size = 28

tokenizer = T5Tokenizer.from_pretrained(model_name)
train_dataset, valid_dataset = create_or_load(tokenizer)

# Run the below cells if you want to train

In [3]:
# if you want to train
class Config:
    # Prompt-tuning
    n_prompt_tokens = 10
    init_from_vocab = True
    # random_range = 0.5
args = Config()

model = T5PromptTuningLM.from_pretrained(
    model_name
    n_tokens=args.n_prompt_tokens,
    initialize_from_vocab=args.init_from_vocab)

Initializing soft prompt...


In [4]:
# Set up training arguments, optimizers, etc
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n == "soft_prompt.weight"],
        "lr": 1e-2,
        "scale_parameter": False,
        "relative_step": False,
    }
]
optimizer = Adafactor(optimizer_grouped_parameters)
lr_scheduler = get_scheduler(
    name='cosine',
    num_warmup_steps=0,
    optimizer=optimizer,
    num_training_steps=3,
)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size*2,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    save_steps=3000,
    report_to='tensorboard',
    prediction_loss_only=True,
    num_train_epochs=3,
)

In [5]:
# Initialize trainer
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=T2TDataCollator(),
        optimizers=(optimizer, lr_scheduler),
    )

In [6]:
# start training
trainer.train()

***** Running training *****
  Num examples = 87599
  Num Epochs = 3
  Instantaneous batch size per device = 28
  Total train batch size (w. parallel, distributed & accumulation) = 28
  Gradient Accumulation steps = 1
  Total optimization steps = 9387


Step,Training Loss
100,1.053200
200,0.959700
300,0.864800
400,0.829000
500,0.789800
600,0.745300
700,0.729700


KeyboardInterrupt: 

In [9]:
# start evaluate, save prompt
trainer.evaluate()
model.save_soft_prompt('soft_prompt', filename=f'soft_prompt_{model_name}_{n_tokens}.model')

# Run the below cells if you want to experiment

In [3]:
# load a new base model with trained soft prompt
model = T5PromptTuningLM.from_pretrained(model_name, 
                                          return_dict=False, 
                                          soft_prompt_path=f'soft_prompt/soft_prompt_{model_name}_{n_tokens}.model')

Set soft prompt! (n_tokens: 10)


In [ ]:
for i in range(len(valid_dataset)):
    print('------------------------------------')
    question, context = valid_dataset['question'][i], valid_dataset['context'][i]
    input_ids = tokenizer.encode('question: %s  context: %s' % (question, context), 
                             return_tensors='pt')
    answers = valid_dataset['answers'][i]['text']
    print(f'context: {context}')
    print()
    print(f'question: {question}')
    print()
    print(f'answers: {answers}')
    encoder_outputs = model.forward(input_ids, to_encoder_only=True)
    indices = model.generate(
        encoder_outputs=encoder_outputs, 
#         repetition_penalty=2.,
                            )
    tokens = indices
    pred = ' '.join([tokenizer.decode(idx, skip_special_tokens=False) for idx in indices])
    print(f'indices: {indices}')
    print(f'model prediction: {pred}')

------------------------------------
context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

question: Which NFL team represented the AFC at Super Bowl 50?

answers: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
indices: tensor([[    0, 12154,     1]])
model pr

indices: tensor([[   0, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,
         2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011]])
model prediction: <pad> Super Super Super Super Super Super Super Super Super Super Super Super Super Super Super Super Super Super Super
------------------------------------
context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game 

indices: tensor([[   0, 1230,    1]])
model prediction: <pad> 2015</s>
------------------------------------
context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

question: What year did the Denver Broncos secure a Super Bowl title for the third time?

answers: ['2015'

indices: tensor([[   0, 5089,    1]])
model prediction: <pad> Carolina</s>
------------------------------------
context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

question: Who won Super Bowl 50?

answers: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
indi

indices: tensor([[   0, 7273,    1]])
model prediction: <pad> 1995</s>
------------------------------------
context: The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl.

question: What team did the Panthers defeat?

answers: ['Arizona Cardinals', 'the Arizona Cardinals', 'Arizona Cardinals']
indices: tensor([[   0, 7601,    1]])
model prediction: <pad> Arizona</s>
------------------------------------
context: T

...

indices: tensor([[   0, 2899,    1]])
model prediction: <pad> 500</s>
------------------------------------
context: The Panthers offense, which led the NFL in scoring (500 points), was loaded with talent, boasting six Pro Bowl selections. Pro Bowl quarterback Cam Newton had one of his best seasons, throwing for 3,837 yards and rushing for 636, while recording a career-high and league-leading 45 total touchdowns (35 passing, 10 rushing), a career-low 10 interceptions, and a career-best quarterback rating of 99.4. Newton's leading receivers were tight end Greg Olsen, who caught a career-high 77 passes for 1,104 yards and seven touchdowns, and wide receiver Ted Ginn, Jr., who caught 44 passes for 739 yards and 10 touchdowns; Ginn also rushed for 60 yards and returned 27 punts for 277 yards. Other key receivers included veteran Jerricho Cotchery (39 receptions for 485 yards), rookie Devin Funchess (31 receptions for 473 yards and five touchdowns), and second-year receiver Corey Brown (31 r

indices: tensor([[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])
model prediction: <pad>
------------------------------------
context: The Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL's active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ sacks, four forced fumbles, and four interceptions, while Kuechly led the team in tackles (118) forced two fumbles, and intercepted four passes of his o

indices: tensor([[  0, 662,   1]])
model prediction: <pad> four</s>
------------------------------------
context: The Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL's active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ sacks, four forced fumbles, and four interceptions, while Kuechly led the team in tackles (118) forced two fumbles, and intercepted four passes of his own. Carolina's secondary featured Pr

indices: tensor([[  0, 662,   1]])
model prediction: <pad> four</s>
------------------------------------
context: Following their loss in the divisional round of the previous season's playoffs, the Denver Broncos underwent numerous coaching changes, including a mutual parting with head coach John Fox (who had won four divisional championships in his four years as Broncos head coach), and the hiring of Gary Kubiak as the new head coach. Under Kubiak, the Broncos planned to install a run-oriented offense with zone blocking to blend in with quarterback Peyton Manning's shotgun passing skills, but struggled with numerous changes and injuries to the offensive line, as well as Manning having his worst statistical season since his rookie year with the Indianapolis Colts in 1998, due to a plantar fasciitis injury in his heel that he had suffered since the summer, and the simple fact that Manning was getting old, as he turned 39 in the 2015 off-season. Although the team had a 7–0 start, Manning

indices: tensor([[    0, 26349,     1]])
model prediction: <pad> Brock</s>
------------------------------------
context: Following their loss in the divisional round of the previous season's playoffs, the Denver Broncos underwent numerous coaching changes, including a mutual parting with head coach John Fox (who had won four divisional championships in his four years as Broncos head coach), and the hiring of Gary Kubiak as the new head coach. Under Kubiak, the Broncos planned to install a run-oriented offense with zone blocking to blend in with quarterback Peyton Manning's shotgun passing skills, but struggled with numerous changes and injuries to the offensive line, as well as Manning having his worst statistical season since his rookie year with the Indianapolis Colts in 1998, due to a plantar fasciitis injury in his heel that he had suffered since the summer, and the simple fact that Manning was getting old, as he turned 39 in the 2015 off-season. Although the team had a 7–0 start, 